In [1]:
# Import necessary libraries
import pandas as pd                # For working with data tables
import geopandas as gpd            # For working with geographic data
from shapely.geometry import Point # For creating point locations on the map
import leafmap                     # For displaying interactive maps
import os
import urllib.request
from urllib.parse import urlparse
import zipfile
import time
import zipfile

# Start the timer for the entire operation
start_time = time.time()

# Create a map centered on the United States, with a zoom level of 4
m = leafmap.Map(center=[37.8, -96.9], zoom=4)

In [2]:
url = "https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip"

# Parse URL and get filename
path = urlparse(url).path
filename = os.path.splitext(os.path.basename(path))[0]

# Create directory
unzip_dir = './extracted_files'
os.makedirs(unzip_dir, exist_ok=True)

zip_path = os.path.join(unzip_dir, f"{filename}.zip")

# Add headers so the server doesn't 500 on default Python UA
headers = {
    "User-Agent": "Mozilla/5.0 (compatible; Python script)",
}

req = urllib.request.Request(url, headers=headers)

# Retry wrapper
retries = 3
for attempt in range(retries):
    try:
        with urllib.request.urlopen(req) as response:
            with open(zip_path, 'wb') as out:
                out.write(response.read())
        print("Downloaded successfully!")
        break
    except Exception as e:
        print(f"Attempt {attempt+1} failed: {e}")
        time.sleep(2)
else:
    raise RuntimeError(f"Failed to download after {retries} attempts")

Downloaded successfully!


In [3]:
# Unzip the downloaded shapefile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

In [4]:
# Load the UniversityDirectory2023 CSV into a data frame
csv_path_university = "./UniversityData/UniversityDirectory2023.csv"
columns_to_use_university = ["UNITID", "INSTNM", "ADDR", "CITY", "STABBR", "LONGITUD", "LATITUDE"]
df_university = pd.read_csv(csv_path_university, usecols=columns_to_use_university, encoding='ISO-8859-1')
print(f"There are {df_university.shape[0]} rows in this data frame!")
df_university.head()

There are 6163 rows in this data frame!


,UNITID,INSTNM,ADDR,CITY,STABBR,LONGITUD,LATITUDE
0,100654,Alabama A & M University,4900 Meridian Street,Normal,AL,-86.568502,34.783368
1,100663,University of Alabama at Birmingham,Administration Bldg Suite 1070,Birmingham,AL,-86.799345,33.505697
2,100690,Amridge University,1200 Taylor Rd,Montgomery,AL,-86.174010,32.362609
3,100706,University of Alabama in Huntsville,301 Sparkman Dr,Huntsville,AL,-86.640449,34.724557
4,100724,Alabama State University,915 S Jackson Street,Montgomery,AL,-86.295677,32.364317


In [5]:
# Load the UniversityAwardedDegrees2023 CSV into a data frame
csv_path_awarded_degrees = "./UniversityData/UniversityAwardedDegrees2023.csv"
df_awarded_degrees = pd.read_csv(csv_path_awarded_degrees, usecols=["UNITID", "CTOTALT"], encoding='ISO-8859-1')
print(f"There are {df_awarded_degrees.shape[0]} rows in this data frame!")
df_awarded_degrees.head()

There are 303292 rows in this data frame!


,UNITID,CTOTALT
0,100654,18
1,100654,8
2,100654,6
3,100654,2
4,100654,2


In [6]:
# Merge the two DataFrames on UNITID
df_merged = pd.merge(df_university, df_awarded_degrees, on="UNITID", how="inner")
print(f"There are {df_merged.shape[0]} rows in this data frame!")
df_merged.head()

There are 303292 rows in this data frame!


,UNITID,INSTNM,ADDR,CITY,STABBR,LONGITUD,LATITUDE,CTOTALT
0,100654,Alabama A & M University,4900 Meridian Street,Normal,AL,-86.568502,34.783368,18
1,100654,Alabama A & M University,4900 Meridian Street,Normal,AL,-86.568502,34.783368,8
2,100654,Alabama A & M University,4900 Meridian Street,Normal,AL,-86.568502,34.783368,6
3,100654,Alabama A & M University,4900 Meridian Street,Normal,AL,-86.568502,34.783368,2
4,100654,Alabama A & M University,4900 Meridian Street,Normal,AL,-86.568502,34.783368,2


In [7]:
df_merged = df_merged.groupby("INSTNM").agg({
    'CTOTALT': 'sum',  # Sum of degrees awarded
    'LATITUDE': 'first',  # You can also use 'mean' or 'median', if preferred
    'LONGITUD': 'first'  # Similarly, 'mean' or 'median' can be used
}).reset_index()
df_merged.head()

,INSTNM,CTOTALT,LATITUDE,LONGITUD
0,A Better U Beauty Barber Academy,78,35.110630,-106.583765
1,A T Still University of Health Sciences,2614,40.193648,-92.589183
2,ABC Adult School,178,33.878179,-118.070114
3,ABC Beauty Academy,104,32.931698,-96.685333
4,ABCO Technology,362,33.932121,-118.369774


In [8]:
# Convert LATITUDE and LONGITUD to geometry points
def make_point(row):
    return Point(row['LONGITUD'], row['LATITUDE'])
df_merged['geometry'] = df_merged.apply(make_point, axis=1)
print(f"There are {df_merged.shape[0]} rows in this data frame!")
df_merged.head()

There are 5864 rows in this data frame!


,INSTNM,CTOTALT,LATITUDE,LONGITUD,geometry
0,A Better U Beauty Barber Academy,78,35.110630,-106.583765,POINT (-106.583765 35.11063)
1,A T Still University of Health Sciences,2614,40.193648,-92.589183,POINT (-92.589183 40.193648)
2,ABC Adult School,178,33.878179,-118.070114,POINT (-118.070114 33.878179)
3,ABC Beauty Academy,104,32.931698,-96.685333,POINT (-96.685333 32.931698)
4,ABCO Technology,362,33.932121,-118.369774,POINT (-118.369774 33.932121)


In [9]:
# Convert the merged DataFrame into a GeoDataFrame
gdf_university = gpd.GeoDataFrame(df_merged, geometry='geometry', crs="EPSG:4326")
gdf_university.head()

,INSTNM,CTOTALT,LATITUDE,LONGITUD,geometry
0,A Better U Beauty Barber Academy,78,35.110630,-106.583765,POINT (-106.58376 35.11063)
1,A T Still University of Health Sciences,2614,40.193648,-92.589183,POINT (-92.58918 40.19365)
2,ABC Adult School,178,33.878179,-118.070114,POINT (-118.07011 33.87818)
3,ABC Beauty Academy,104,32.931698,-96.685333,POINT (-96.68533 32.9317)
4,ABCO Technology,362,33.932121,-118.369774,POINT (-118.36977 33.93212)


In [10]:
# WARNING - THIS CAN TAKE A LONG TIME DEPENDING ON THE SPECS OF YOUR PC
# Add the university point layer to the map
m.add_gdf(gdf_university, layer_name="All Universities")

In [11]:
# WARNING - THIS CAN TAKE A LONG TIME DEPENDING ON THE SPECS OF YOUR PC
# Load the Shapefile for US States from the correct directory
shapefile_path = "./extracted_files/cb_2018_us_state_500k.shp"
gdf_states = gpd.read_file(shapefile_path)
gdf_states.head()

# Add the states with the degree totals to the map, symbolizing by the 'Total degrees awarded' column
m.add_gdf(gdf_states, layer_name="States")
m

Map(center=[37.8, -96.9], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [ ]:
# Start the timer for the spatial join
spatial_join_start_time = time.time()

# Perform a spatial join between the university data and the state shapefile
gdf_joined = gpd.sjoin(gdf_university, gdf_states, how="inner", predicate="within")

# Calculate and print the time taken for the spatial join
spatial_join_end_time = time.time()
spatial_join_duration = spatial_join_end_time - spatial_join_start_time
print(f"Time taken for spatial join: {spatial_join_duration:.2f} seconds")


In [ ]:
# Aggregate the joined data by state and sum the degrees awarded
gdf_state_degrees = gdf_joined.groupby("STUSPS")["CTOTALT"].sum().reset_index()

# Merge the aggregated data back into the state boundaries GeoDataFrame
gdf_states = gdf_states.merge(gdf_state_degrees, left_on="STUSPS", right_on="STUSPS", how="left")
print(gdf_states.head())

In [ ]:
# Use the 'Name' from the shapefile for the full state names and clean up the DataFrame
gdf_states["State Name"] = gdf_states["NAME"]
gdf_states["State Abbreviation"] = gdf_states["STUSPS"]
gdf_states["Total degrees awarded"] = gdf_states["CTOTALT"]

# Drop unnecessary columns and keep the relevant ones
gdf_states = gdf_states[["State Name", "State Abbreviation", "Total degrees awarded", "geometry"]]

# Add the states with the degree totals to the map, symbolizing by the 'Total degrees awarded' column
m.add_gdf(gdf_states, 
          layer_name="States with Degrees",
          color_by="Total degrees awarded",
          color_scale="YlOrRd",
         )

# Drop unnecessary columns and keep the relevant ones
#gdf_states = gdf_states[["State Name", "State Abbreviation", "Total degrees awarded"]]

m.add_labels(
    gdf_states,
    column="Total degrees awarded",
    label_font_size=12,
    label_color="black",
    label_offset=[0, 0],
    layer_name="Degree Labels"
)

# Show the map
m

In [ ]:
# Generate some interesting facts
num_rows, num_columns = df_merged.shape
num_cells = num_rows * num_columns

print(f"Total number of rows: {num_rows:,}")
print(f"Total number of columns: {num_columns:,}")
print(f"Total number of cells: {num_cells:,}")

In [ ]:
# Calculate the sum of all degrees awarded across all universities
total_degrees_awarded = gdf_states["Total degrees awarded"].sum()

# Print the result with commas for readability
print(f"Total degrees awarded across all states: {total_degrees_awarded:,}")

In [ ]:
# Complete the full operation
end_time = time.time()
total_duration = end_time - start_time

# Print the total operation time and format with commas
print(f"Total operation time: {total_duration:.2f} seconds")

In [ ]:
# Reproject the GeoDataFrame to EPSG:4326
gdf_states = gdf_states.to_crs(epsg=4326)

# Export the GeoDataFrame to GeoJSON without the CRS field
gdf_states.to_file("states_degrees_awarded.geojson", driver="GeoJSON")

In [ ]:
# Manually upload your GeoJSON to ArcGIS Online: https://www.arcgis.com/home/content.html

In [ ]:
# Import that layer into a Map: https://www.arcgis.com/apps/mapviewer/index.html
# Make sure to SAVE!

In [ ]:
# Can you repeat thsi exercise using US counties?!
# https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_500k.zip